In [37]:
import pandas as pd
import numpy as np
from collections import Counter
import random

In [38]:
df = pd.read_csv('FY_2025_Hospital_Readmissions_Reduction_Program_Hospital.csv')
print("Initial data shape:", df.shape)
print("\n\n", df.head())
print("\n\n", df.dtypes)

Initial data shape: (18510, 12)


                      Facility Name  Facility ID State            Measure Name  \
0  SOUTHEAST HEALTH MEDICAL CENTER        10001    AL       READM-30-AMI-HRRP   
1  SOUTHEAST HEALTH MEDICAL CENTER        10001    AL      READM-30-CABG-HRRP   
2  SOUTHEAST HEALTH MEDICAL CENTER        10001    AL        READM-30-HF-HRRP   
3  SOUTHEAST HEALTH MEDICAL CENTER        10001    AL  READM-30-HIP-KNEE-HRRP   
4  SOUTHEAST HEALTH MEDICAL CENTER        10001    AL        READM-30-PN-HRRP   

   Number of Discharges  Footnote  Excess Readmission Ratio  \
0                 296.0       NaN                    0.9483   
1                 151.0       NaN                    0.9509   
2                 681.0       NaN                    1.0597   
3                   NaN       NaN                    0.9654   
4                 490.0       NaN                    0.9715   

   Predicted Readmission Rate  Expected Readmission Rate  \
0                     13.0146          

In [39]:
# CLEANING OF DATA
df['Number of Readmissions'] = pd.to_numeric(df['Number of Readmissions'], errors='coerce')

# Drop 'Number of Readmissions' that have 'Too Few to Report" and empty 'Excess Readmission ratio' 
df = df.dropna(subset=['Number of Readmissions'])
df = df.dropna(subset=['Excess Readmission Ratio'])

# Filling in empty comlumns with set string
df['Footnote'] = df['Footnote'].fillna('No notes')

# Have to encode all categorical to numbers for random forest
categorical_cols = df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_cols:
    df[col] = df[col].astype(str)
    unique_values = df[col].unique()
    encoder = {val: idx for idx, val in enumerate(unique_values)}
    df[col] = df[col].map(encoder)
    label_encoders[col] = encoder

# Drop dates since its the same time period
df = df.drop(columns=['Start Date', 'End Date'])

In [40]:
print("Post Cleanse data shape:", df.shape)
print("\n\n", df.head())
print("\n\n", df.dtypes)

Post Cleanse data shape: (8121, 10)


    Facility Name  Facility ID  State  Measure Name  Number of Discharges  \
0              0        10001      0             0                 296.0   
1              0        10001      0             1                 151.0   
2              0        10001      0             2                 681.0   
4              0        10001      0             3                 490.0   
5              0        10001      0             4                 130.0   

   Footnote  Excess Readmission Ratio  Predicted Readmission Rate  \
0         0                    0.9483                     13.0146   
1         0                    0.9509                      9.6899   
2         0                    1.0597                     21.5645   
4         0                    0.9715                     16.1137   
5         0                    0.9330                     15.4544   

   Expected Readmission Rate  Number of Readmissions  
0                    13.7235      

In [41]:
# Targetting high cost risk facilities where readmission ratio is greater than 1
df['High_Cost_Risk'] = (df['Excess Readmission Ratio'] > 1).astype(int)

X = df.drop(columns=['High_Cost_Risk']).values
y = df['High_Cost_Risk'].values


In [42]:
class Node:
    def __init__(self, feature_idx=None, threshold=None, left=None, right=None, *, value=None):
        self.feature_idx = feature_idx
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

# Recursively building the decision tree
def build_tree(X, y, depth=0, max_depth=10):
    if len(set(y)) == 1 or depth >= max_depth:
        leaf_value = Counter(y).most_common(1)[0][0]
        return Node(value=leaf_value)

    best_feat, best_thresh = best_split(X, y)

    if best_feat is None:
        leaf_value = Counter(y).most_common(1)[0][0]
        return Node(value=leaf_value)

    X_left, y_left, X_right, y_right = split_dataset(X, y, best_feat, best_thresh)

    left_child = build_tree(X_left, y_left, depth + 1, max_depth)
    right_child = build_tree(X_right, y_right, depth + 1, max_depth)

    return Node(feature_idx=best_feat, threshold=best_thresh, left=left_child, right=right_child)

def predict_tree(x, tree):
    if tree.is_leaf_node():
        return tree.value
    if x[tree.feature_idx] <= tree.threshold:
        return predict_tree(x, tree.left)
    else:
        return predict_tree(x, tree.right)

def predict(X, tree):
    return np.array([predict_tree(x, tree) for x in X])

def gini(y):
    counts = np.bincount(y)
    probabilities = counts / len(y)
    return 1 - np.sum(probabilities ** 2)

def split_dataset(X, y, feature_idx, threshold):
    left_indices = X[:, feature_idx] <= threshold
    right_indices = X[:, feature_idx] > threshold
    return X[left_indices], y[left_indices], X[right_indices], y[right_indices]

def best_split(X, y):
    best_gain = -1
    split_idx, split_threshold = None, None
    n_features = X.shape[1]

    for feature_idx in range(n_features):
        values = np.unique(X[:, feature_idx])
        for threshold in values:
            X_left, y_left, X_right, y_right = split_dataset(X, y, feature_idx, threshold)
            if len(y_left) == 0 or len(y_right) == 0:
                continue

            p = len(y_left) / len(y)
            gain = gini(y) - (p * gini(y_left) + (1 - p) * gini(y_right))

            if gain > best_gain:
                best_gain = gain
                split_idx = feature_idx
                split_threshold = threshold

    return split_idx, split_threshold

In [43]:
def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

def confusion_matrix(y_true, y_pred):
    unique_classes = np.unique(y_true)
    matrix = np.zeros((len(unique_classes), len(unique_classes)), dtype=int)

    for true_label, pred_label in zip(y_true, y_pred):
        matrix[true_label][pred_label] += 1

    return matrix

In [44]:
tree = build_tree(X, y, max_depth=10)

predictions = predict(X, tree)

print(f"\nModel Accuracy: {accuracy(y, predictions):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix_manual(y, predictions))


Model Accuracy: 1.0000

Confusion Matrix:
[[3667    0]
 [   0 4454]]


In [45]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix


kf = KFold(n_splits=5, shuffle=True, random_state=40)
fold = 1
accuracies = []
all_y_true = []
all_y_pred = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    tree = build_tree(X_train, y_train, max_depth=10)

    y_pred = predict(X_test, tree)

    acc = accuracy(y_test, y_pred)
    accuracies.append(acc)

    print(f"Fold {fold} Accuracy: {acc:.4f}")
    fold += 1

    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

# metrics from validation
print(f"\nAverage K-Fold Accuracy: {np.mean(accuracies):.4f}")

print("\n\n\nClassification Report (across all folds):")
print(classification_report(all_y_true, all_y_pred))

print("\n\n\nOverall Confusion Matrix:")
print(confusion_matrix(all_y_true, all_y_pred))

Fold 1 Accuracy: 1.0000
Fold 2 Accuracy: 1.0000
Fold 3 Accuracy: 1.0000
Fold 4 Accuracy: 1.0000
Fold 5 Accuracy: 1.0000

Average K-Fold Accuracy: 1.0000



Classification Report (across all folds):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3667
           1       1.00      1.00      1.00      4454

    accuracy                           1.00      8121
   macro avg       1.00      1.00      1.00      8121
weighted avg       1.00      1.00      1.00      8121




Overall Confusion Matrix:
[[3667    0]
 [   0 4454]]
